# Prepare the Web Driver

The tutorial is from https://towardsdatascience.com/image-scraping-with-python-a96feda8af2d

In [104]:
from selenium import webdriver
# This is the path I use
# DRIVER_PATH = '.../Desktop/Scraping/chromedriver 2'
# Put the path for your ChromeDriver here
# If this error occurs: WebDriverException: Message: 'Webdrivers' executable may have wrong permissions.
## See this post: https://stackoverflow.com/questions/47148872/webdrivers-executable-may-have-wrong-permissions-please-see-https-sites-goo

DRIVER_PATH = '/home/shin/Bangkit Academy/CAPSTONE PROJECT/chromedriver_linux64/chromedriver'
wd = webdriver.Chrome(executable_path=DRIVER_PATH)

Scrap Informations from URLs

In [105]:
import time

def fetch_infos(queries:str, wd:webdriver):
    infos = []
    
    for query in queries:
        # build the google query
        search_url = "https://google.com/search?safe=off&site=&q={q}&language=id"

        # load the page
        wd.get(search_url.format(q=query))


        info = {}
        try:
            infobox = wd.find_elements_by_id("wp-tabs-container")[0]
        
            # title
            title = infobox.find_elements_by_tag_name('h2')[0]
            info['title'] = title.text

            # description
            desc = infobox.find_elements_by_class_name('kno-rdesc')[0].find_elements_by_tag_name('span')[0]
            info['desc'] = desc.text

            # scientific name
            sn = infobox.find_elements_by_class_name('kno-fv')[0]
            info['scientific_name'] = sn.text

            # source
            src = infobox.find_elements_by_class_name('fl')[0]
            info['source'] = src.get_attribute('href')

            # images sample
            num = 4
            img_urls = []
            imgs = infobox.find_elements_by_tag_name('img')
            for im in imgs[:num]:
                link = im.get_attribute('src')
                img_urls.append(link)
            info['img_urls'] = img_urls

        except:
            pass

        infos.append(info)
        time.sleep(1)

        print(f'DONE: {query}')

    return infos

## Read list of keywords to search from file

In [106]:
lines=[]
with open('nama_tumbuhan.txt', 'r') as f:
    lines = f.readlines()

queries = list(map(lambda x: x.strip(), lines))
infos = fetch_infos(queries, wd)



DONE: anggrek
DONE: mawar
DONE: melati
DONE: mangga
DONE: kelapa
DONE: singkong
DONE: ubi jalar
DONE: tebu
DONE: padi
DONE: bayam
DONE: jambu
DONE: teratai
DONE: lidah buaya
DONE: bengkuang
DONE: ceremai
DONE: cempaka
DONE: ciplukan
DONE: durian
DONE: mengkudu
DONE: sirih
DONE: gambir
DONE: alpukat
DONE: apel
DONE: maja
DONE: jati
DONE: karet
DONE: kenanga
DONE: kapuk randu
DONE: gaharu
DONE: matoa
DONE: nibung
DONE: delima
DONE: belimbing
DONE: jeruk
DONE: manggis
DONE: pisang
DONE: nangka
DONE: rambutan
DONE: pepaya
DONE: sawo
DONE: cabai
DONE: cengkeh
DONE: kencur
DONE: jahe
DONE: lada
DONE: pala
DONE: kentang
DONE: terung
DONE: kacang tanah
DONE: kacang hijau


In [109]:
keys = ['title', 'desc', 'scientific_name']
with open('logs', 'w') as f:
    for info in infos:
        for key in keys:
            if key in info:
                f.writeln(info[key])
            else:
                print(f'[x] no {key}')
        if 'img_urls' in info:
            f.writeln(len(info['img_urls']))
        else:
            f.writeln('[x] no images')
        f.writeln('----')

AttributeError: '_io.TextIOWrapper' object has no attribute 'writeline'

# Save Images

In [94]:
import base64
import os

def save_images(urls:str, folder:str):
    try:
        os.mkdir(folder)
    except:
        pass
    for i, url in enumerate(urls):
        head, data = url.split(',')
        image_data = base64.b64decode(data)

        file_path = os.path.join(folder, f'{i}.jpg')
        with open(file_path, 'wb') as f:
            f.write(image_data)

In [98]:
parent_dir = 'wiki_images'
for i, item in enumerate(infos):
    try:
        title = item['title']
        imgs = item['img_urls']
        folder = os.path.join(parent_dir, title)
        save_images(imgs, folder)
    except Exception as e:
        print(f'ERROR at index {i} : {e}')

ERROR at index 4 : 'img_urls'
ERROR at index 7 : 'img_urls'
ERROR at index 8 : not enough values to unpack (expected 2, got 1)
ERROR at index 12 : not enough values to unpack (expected 2, got 1)
ERROR at index 15 : 'img_urls'
ERROR at index 19 : not enough values to unpack (expected 2, got 1)
ERROR at index 23 : not enough values to unpack (expected 2, got 1)
ERROR at index 25 : 'img_urls'
ERROR at index 28 : 'img_urls'
ERROR at index 35 : 'img_urls'
ERROR at index 40 : 'img_urls'
ERROR at index 45 : 'img_urls'
ERROR at index 46 : 'title'
ERROR at index 48 : 'img_urls'


# Save the Information

In [ ]:
with open('info.json', 'w') as f:
    for info in infos:
        

# Close the Web Driver

In [ ]:
wd.close()